In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train_data=open("/content/train14.txt",mode='r').readlines()
val_data=open("/content/val.txt", mode='r').readlines()
test_data=open("/content/test.txt",mode='r').readlines()

In [6]:
import re
def data_preprocessing(dataset):
    text=[]
    mood=[]
    for i in range(len(dataset)):
        a1=dataset[i].split(";")[0]
        a2=dataset[i].split(";")[1]
        a3=re.findall("^\w+",a2)[0]
        text.append(a1)
        mood.append(a3)
    return  text ,mood

In [7]:
train_text,train_mood=data_preprocessing(train_data)
print("******")
print(len(train_text))
print("*******")
print(len(train_mood))
print("*****")
print(set(train_mood))

******
16000
*******
16000
*****
{'joy', 'sadness', 'anger', 'fear', 'love', 'surprise'}


In [8]:
val_text,val_mood=data_preprocessing(val_data)
print("********")
print(len(val_text))
print("*******")
print(len(val_mood))
print("******")
print(set(val_mood))

********
2000
*******
2000
******
{'love', 'sadness', 'anger', 'fear', 'joy', 'surprise'}


In [9]:
test_text,test_mood=data_preprocessing(test_data)
print("********")
print(len(test_text))
print("*******")
print(len(test_mood))
print("******")
print(set(test_mood))

********
2000
*******
2000
******
{'joy', 'sadness', 'anger', 'fear', 'love', 'surprise'}


In [21]:
#sentiment #Joy, Sad,Happy, Anger, Frustration
#1 = positive sentiment
#0 = negative sentiemnt
def label_create(moods):
    labels=[]
    for i in moods:
        if i=="Joy"or i=="love" or i=="surprise":
            labels.append(1)
        else:
            labels.append(0)
    return labels


In [11]:
train_labels=label_create(train_mood)
print(len(train_labels))
print("********")
val_labels=label_create(val_mood)
print(len(val_labels))
print("********")
test_labels=label_create(test_mood)
print(len(test_labels))

16000
********
2000
********
2000


In [12]:
train_id=[]
val_id=[]
test_id=[]
for i in range(0,len(train_data)):
    train_id.append(i)
for j in range(len(train_data),len(train_data)+len(val_data)):
    val_id.append(j)
for k in range(len(train_data)+len(val_data),len(train_data)+len(val_data)+len(test_data)):
    test_id.append(k)

In [13]:
print(len(train_id))
print("*******")
print(len(val_id))
print("*********")
print(len(test_id))
print("*******")

16000
*******
2000
*********
2000
*******


In [28]:
train_data1=pd.DataFrame(list(zip(train_id,train_text,train_mood)),columns=["ID","text","mood"])
print(train_data1.shape)
print(train_data1.head(5))

(16000, 3)
   ID                                               text     mood
0   0                            i didnt feel humiliated  sadness
1   1  i can go from feeling so hopeless to so damned...  sadness
2   2   im grabbing a minute to post i feel greedy wrong    anger
3   3  i am ever feeling nostalgic about the fireplac...     love
4   4                               i am feeling grouchy    anger


In [27]:
val_data1=pd.DataFrame(list(zip(val_id,val_text,val_mood)),columns=["ID","text","mood"])
print(val_data1.shape)
print(val_data1.head(5))

(2000, 3)
      ID                                               text     mood
0  16000  im feeling quite sad and sorry for myself but ...  sadness
1  16001  i feel like i am still looking at a blank canv...  sadness
2  16002                     i feel like a faithful servant     love
3  16003                  i am just feeling cranky and blue    anger
4  16004  i can have for a treat or if i am feeling festive      joy


In [29]:
test_data1=pd.DataFrame(list(zip(test_id,test_text,test_mood)),columns=["ID","text","mood"])
print(test_data1.shape)
print(test_data1.head(5))

(2000, 3)
      ID                                               text     mood
0  18000  im feeling rather rotten so im not very ambiti...  sadness
1  18001          im updating my blog because i feel shitty  sadness
2  18002  i never make her separate from me because i do...  sadness
3  18003  i left with my bouquet of red and yellow tulip...      joy
4  18004    i was feeling a little vain when i did this one  sadness


In [20]:
test_labels[0]

0

#vectorizing text data

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [31]:
tokenizer=Tokenizer(oov_token="<oov>")

In [32]:
tokenizer.fit_on_texts(train_text)
word_index=tokenizer.word_index

In [33]:
len(word_index)

15213

In [34]:
x_train_sequence=tokenizer.texts_to_sequences(train_text)
x_val_sequence=tokenizer.texts_to_sequences(val_text)
x_test_sequence=tokenizer.texts_to_sequences(test_text)

In [35]:
len11=[] # observing what are the lengths of sequence and according to that will select maxlen
for i in x_train_sequence:
    len11.append(len(i))

In [37]:
print(len(len11))

16000


In [38]:
max(len11)

66

In [39]:
np.percentile(len11,90) #90 percentile value of different lengths

35.0

In [40]:
print(np.arange(0,110,10))
print("***********")
print(np.percentile(len11,np.arange(0,110,10)))

[  0  10  20  30  40  50  60  70  80  90 100]
***********
[ 2.  7.  9. 12. 14. 17. 20. 23. 28. 35. 66.]


In [41]:
print(np.percentile(len11,np.arange(90,101,1)))

[35. 36. 37. 38. 39. 41. 43. 45. 48. 52. 66.]


#99.99% of the texts have 52 as there length so we will take it as it is

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_padded=pad_sequences(x_train_sequence,maxlen=52,padding='post',truncating="post")
x_val_padded=pad_sequences(x_val_sequence,maxlen=52,padding='post',truncating="post")
x_test_padded=pad_sequences(x_test_sequence,maxlen=52,padding='post',truncating='post')

In [43]:
print(x_train_padded.shape)
print("********")
print(x_val_padded.shape)
print("*********")
print(x_test_padded.shape)

(16000, 52)
********
(2000, 52)
*********
(2000, 52)


#encoding numerical feature

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
vectorizer_mood =  CountVectorizer()
vectorizer_mood.fit(train_data1["mood"].values) #fit has to happen only on train data

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [49]:
train_vectorizer_mood=vectorizer_mood.transform(train_data1["mood"].values)
train_vectorizer_mood.shape

(16000, 6)

In [50]:
val_vectorizer_mood=vectorizer_mood.transform(val_data1["mood"].values)
test_vectorizer_mood=vectorizer_mood.transform(test_data1["mood"].values)

In [51]:
print(val_vectorizer_mood.shape)
print("******")
print(test_vectorizer_mood.shape)

(2000, 6)
******
(2000, 6)


In [88]:
from scipy.sparse import hstack
X_tr=hstack((x_train_padded,train_vectorizer_mood)).todense()
X_tr.shape

(16000, 58)

In [89]:
X_val=hstack((x_val_padded,val_vectorizer_mood)).todense()
print(X_val.shape)
print("******")
X_test=hstack((x_test_padded,test_vectorizer_mood)).todense()
print(X_test.shape)

(2000, 58)
******
(2000, 58)


In [90]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [91]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

In [92]:
print(type(X_tr))

<class 'numpy.matrix'>


In [93]:
y_train=pd.DataFrame(train_labels)
y_val=pd.DataFrame(val_labels)
y_test=pd.DataFrame(test_labels)

In [94]:
history = model.fit(X_tr,y_train,epochs=50,batch_size=32,validation_data=(X_val,y_val))

Epoch 1/50
500/500 [==============================] - 2s 2ms/step - loss: 26.9474 - accuracy: 0.7832 - val_loss: 13.4447 - val_accuracy: 0.8205
Epoch 2/50
500/500 [==============================] - 1s 2ms/step - loss: 11.7306 - accuracy: 0.7842 - val_loss: 9.5009 - val_accuracy: 0.8415
Epoch 3/50
500/500 [==============================] - 1s 2ms/step - loss: 7.4295 - accuracy: 0.7972 - val_loss: 6.4345 - val_accuracy: 0.8280
Epoch 4/50
500/500 [==============================] - 1s 2ms/step - loss: 5.1776 - accuracy: 0.8015 - val_loss: 4.4231 - val_accuracy: 0.7915
Epoch 5/50
500/500 [==============================] - 1s 2ms/step - loss: 3.6769 - accuracy: 0.8124 - val_loss: 4.2532 - val_accuracy: 0.8525
Epoch 6/50
500/500 [==============================] - 1s 2ms/step - loss: 2.7754 - accuracy: 0.8127 - val_loss: 2.9318 - val_accuracy: 0.8450
Epoch 7/50
500/500 [==============================] - 1s 2ms/step - loss: 2.0696 - accuracy: 0.8201 - val_loss: 2.0623 - val_accuracy: 0.7835
Epo

In [97]:
results = model.evaluate(X_test, y_test)
results

63/63 [==============================] - 0s 1ms/step - loss: 0.4730 - accuracy: 0.8880


[0.4730384647846222, 0.8880000114440918]